In [44]:
import pyodbc
server = 'LAPTOP-G2C53J66'
database = 'arcane_vault'
username = 'admin'
password = 'admin'

conn = pyodbc.connect(
    'DRIVER={SQL Server};' +
    'SERVER='+ server + ';' +
    'DATABASE=' + database + ';' +
    'UID=' + username + ';' +
    'PWD=' + password)

cursor = conn.cursor()

adventurers_path = 'D:\\Projekty\\Hurtownie Danych\\fantasy-data-generator\\data\\adventurers_t2.csv'

adventures_path = 'D:\\Projekty\\Hurtownie Danych\\fantasy-data-generator\\data\\adventures_t2.csv'

artifacts_path = 'D:\\Projekty\\Hurtownie Danych\\fantasy-data-generator\\data\\artifacts_t2.csv'

participations_path = 'D:\\Projekty\\Hurtownie Danych\\fantasy-data-generator\\data\\participations_t2.csv'

In [45]:
create_temp_adventurers_query = '''
CREATE TABLE tempAdventurers(
    adventurer_id  INT PRIMARY KEY,
    name NVARCHAR(20),
    surname NVARCHAR(30),
    level TINYINT CHECK (level >= 1 AND level <=20),
)
'''

cursor.execute(create_temp_adventurers_query)
conn.commit()

adventurers_insert_query = f'''
BULK INSERT tempAdventurers
FROM '{adventurers_path}'
WITH (
    FIRSTROW=2,
    FIELDTERMINATOR=',',
    ROWTERMINATOR='0x0a'
)
'''

cursor.execute(adventurers_insert_query)
conn.commit()

merge_adventurers_query = '''
MERGE INTO adventurers AS Target
USING tempAdventurers AS Source
ON Target.adventurer_id = Source.adventurer_id

WHEN MATCHED THEN
    UPDATE SET
    Target.name = Source.name,
    Target.surname = Source.surname,
    Target.level = Source.level

WHEN NOT MATCHED BY TARGET THEN
    INSERT (adventurer_id, name, surname, level)
    VALUES (Source.adventurer_id, Source.name, Source.surname, Source.level)
    ;
'''

cursor.execute(merge_adventurers_query)
conn.commit()

drop_temp_adventurers_query = '''
DROP TABLE tempAdventurers
'''

cursor.execute(drop_temp_adventurers_query)
conn.commit()

In [46]:
create_temp_adventures_query = '''
CREATE TABLE tempAdventures(
    adventure_id INT PRIMARY KEY,
    facility_id INT,
    destination_id INT,
    date DATE,
    deadline DATE,
    status NVARCHAR(30) CHECK (status IN ('planned', 'in progress', 'finished', 'failed')),
    cost INT,
)
'''

cursor.execute(create_temp_adventures_query)
conn.commit()

adventures_insert_query = f'''
BULK INSERT tempAdventures
FROM '{adventures_path}'
WITH (
    FIRSTROW=2,
    FIELDTERMINATOR=',',
    ROWTERMINATOR='0x0a'
)
'''

cursor.execute(adventures_insert_query)
conn.commit()

merge_adventures_query = '''
MERGE INTO adventures AS Target
USING tempAdventures AS Source
ON Target.adventure_id = Source.adventure_id

WHEN MATCHED THEN
    UPDATE SET
    Target.date = Source.date,
    Target.deadline = Source.deadline,
    Target.status = Source.status,
    Target.cost = Source.cost

WHEN NOT MATCHED BY TARGET THEN
    INSERT (adventure_id, facility_id, destination_id, date, deadline, status, cost)
    VALUES (Source.adventure_id, Source.facility_id, Source.destination_id, Source.date, Source.deadline, Source.status, Source.cost)
    ;
'''

cursor.execute(merge_adventures_query)
conn.commit()

drop_temp_adventures_query = '''
DROP TABLE tempAdventures
'''

cursor.execute(drop_temp_adventures_query)
conn.commit()

In [47]:
create_temp_artifacts_query = '''
CREATE TABLE tempArtifacts(
    artifact_id INT PRIMARY KEY,
    adventure_id INT,
    status NVARCHAR(30) CHECK (status IN ('Unidentified', 'Identified', 'Sold', 'Stolen', 'Damaged')),
    kind NVARCHAR(50),
);
'''

cursor.execute(create_temp_artifacts_query)
conn.commit()

artifacts_insert_query = f'''
BULK INSERT tempArtifacts
FROM '{artifacts_path}'
WITH (
    FIRSTROW=2,
    FIELDTERMINATOR=',',
    ROWTERMINATOR='0x0a'
)
'''

cursor.execute(artifacts_insert_query)
conn.commit()

merge_artifacts_query = '''
MERGE INTO artifacts AS Target
USING tempArtifacts AS Source
ON Target.artifact_id = Source.artifact_id

WHEN MATCHED THEN
    UPDATE SET
    Target.status = Source.status,
    Target.kind = Source.kind

WHEN NOT MATCHED BY TARGET THEN
    INSERT (artifact_id, adventure_id, status, kind)
    VALUES (Source.artifact_id, Source.adventure_id, Source.status, Source.kind)
    ;
'''

cursor.execute(merge_artifacts_query)
conn.commit()

drop_temp_artifacts_query = '''
DROP TABLE tempArtifacts
'''

cursor.execute(drop_temp_artifacts_query)
conn.commit()

In [48]:
create_temp_participations_query = '''
CREATE TABLE tempParticipations(
    adventurer_id INT,
    adventure_id INT,
    PRIMARY KEY (adventurer_id, adventure_id)
);
'''

cursor.execute(create_temp_participations_query)
conn.commit()

participations_insert_query = f'''
BULK INSERT tempParticipations
FROM '{participations_path}'
WITH (
    FIRSTROW=2,
    FIELDTERMINATOR=',',
    ROWTERMINATOR='0x0a'
)
'''

cursor.execute(participations_insert_query)
conn.commit()

merge_participations_query = '''
MERGE INTO participations AS Target
USING tempParticipations AS Source
ON Target.adventurer_id = Source.adventurer_id AND Target.adventure_id = Source.adventure_id

WHEN NOT MATCHED BY TARGET THEN
    INSERT (adventurer_id, adventure_id)
    VALUES (Source.adventurer_id, Source.adventure_id)
    ;
'''

cursor.execute(merge_participations_query)
conn.commit()

drop_temp_participations_query = '''
DROP TABLE tempParticipations
'''

cursor.execute(drop_temp_participations_query)
conn.commit()

In [49]:
conn.close()